Zastosowane 4 metody dra Kontka dla danych z 2020 roku (Duda & Trzaskowski)

In [1]:
import pandas as pd
import re
import os
import numpy as np
from scipy.stats import median_abs_deviation

from src.utilities import presidential_data
from src.utilities import kontek_clustering as clustering
from src.utilities import kontek_methods as methods

### load data

In [2]:
year = "2020"
df_2020_r1 = presidential_data.get_df(year, "1")
df_2020_r2 = presidential_data.get_df(year, "2")

------------

### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [3]:
cand_A = "trzaskowski"
cand_B = "duda"

df_2020 = presidential_data.join_both_rounds(cand_A, cand_B, df_2020_r1, df_2020_r2)

In [4]:
# Add buckets
df_2020 = clustering.add_bialek_postal_buckets(df_2020)

### 1. Nadmierne poparcie dla Andrzeja Dudy (względem mediany w ramach lokalnej grupy)

In [15]:
df = df_2020.copy()

2.4 Przeliczenie wyników

Aby uwzględnić niepewność i wrażliwość zastosowanego podejścia, obliczenia przeprowadzono dla
trzech różnych progów detekcji wartości odstających: k > 2.0, k > 2.5 oraz k > 3.0, gdzie k oznacza
liczbę jednostek odchylenia bezwzględnego od mediany (MAD) względem mediany w grupie
lokalnej. Wyższe wartości k wyodrębniają jedynie najbardziej skrajne przypadki, zapewniając tym
samym konserwatywną estymację potencjalnego wpływu. Jednocześnie jednak ograniczają zdolność
metody do wychwytywania mniejszych, lecz wciąż istotnych odchyleń.

In [6]:
# Let's set k = 2
k = 2

In [16]:
df = methods.add_anomaly_1(df, cand_A, cand_B, k=k, new_col_name="anomaly_1")

In [17]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
trzaskowski: 3430
duda: 2893
---
k > 2.5
trzaskowski: 2541
duda: 2018
---
k > 3.0
trzaskowski: 1917
duda: 1434
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Duda jest 2893  a Trzaskowski 3430 (vs 3750 i 4551 w 2025)

In [19]:
# drop k_score columns
# should be done in add_anomaly func, but to show different anomaly results based on k

df = df.drop(columns=[
    f"{cand_A}_k_score_1",
    f"{cand_B}_k_score_1"
])

### 2. Nadmierny względny wzrost poparcia dla Andrzeja Dudy między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;

za JB: 

Nie podano wprost jak to było obliczone więc kolejno:

  1. Dla danego kandydata obliczam względny wzrost między pierwszą a drugą turą (dzieląc wynik z drugiej przez wynik z pierwszej)
  2. Następnie odnoszę go do wzrostu drugiego kandydata - liczę różnicę między względnymi wzrostami.
  3. Dalej tak jak w pierwszym typie anomalii - dla tych różnic liczę medianę grupy, MAD grupy oraz odchylenie k w komisji.

In [20]:
df = methods.add_anomaly_2(df, cand_A, cand_B, k=k, new_col_name="anomaly_2")

In [21]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
trzaskowski: 3795
duda: 1863
---
k > 2.5
trzaskowski: 2938
duda: 1235
---
k > 3.0
trzaskowski: 2358
duda: 865
---


vs 2025:

k > 2.0

trzaskowski: 3552

nawrocki: 3127



In [22]:
df = df.drop(columns=[
    f"{cand_A}_k_score_2",
    f"{cand_B}_k_score_2"
])

### 3. Komisje, w których Duda uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

  1. Sprawdzamy, w których grupach dany kandydat miał większą medianę
  2. Sumujemy komisje, w których wygrał kandydat A mimo, że większą medianę miał kandydat B i na odwrót.

In [23]:
df = methods.add_anomaly_3(df, cand_A, cand_B, k=2, new_col_name="anomaly_3")

In [24]:
# na korzyść kandydat A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 1885)

vs 2025:

('trzaskowski', 2608)

In [25]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('duda', 1999)

vs 2025:

('nawrocki', 1843)

**WYNIKI**:

W grupach, w których większą medianę miał Duda, było 1999 (2025 - 1843) komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1885 (2025 - 2608) komisji, w których wyższy wyniki uzyskał Nawrocki.


In [27]:
df = df.drop(columns=[
    f"higher_median_{cand_A}",
    f"higher_median_{cand_B}"	
])

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [28]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 106)

vs 2025:

('trzaskowski', 128)

W 106 (2025 - 128) komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [29]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('duda', 284)

vs 2025:

('nawrocki', 112)

W 284 (2025 - 112) komisjach Duda uzyskał mniej głosów w drugiej turze niż w pierwszej.


Przykładowe anomalie na korzyść Trzaskowskiego:

In [30]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,trzaskowski_r1,duda_r1,postal_code,trzaskowski_r2,duda_r2,postal_clean,bucket,trzaskowski_anomaly_1,duda_anomaly_1,trzaskowski_anomaly_2,duda_anomaly_2,trzaskowski_anomaly_3,duda_anomaly_3
10728,140611,6,287,684,05-660,842,506,05660,94,True,False,True,False,True,False
21288,247101,16,160,493,41-940,483,447,41940,1081,False,False,True,False,False,False
5955,81006,1,125,402,67-312,488,220,67312,2377,True,False,True,False,True,False
24593,301302,1,90,360,64-111,289,339,64111,1497,False,False,True,False,False,False
13392,146507,430,576,307,04-173,854,299,04173,27,False,False,True,False,False,False


In [31]:
# Anomalie na korzysc
df[cand_A + '_anomaly_4'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_anomaly_4'] = df[cand_A + '_r2']<df[cand_A + '_r1']

To są rzeczywiście bardzo podejrzane przypadki i o takich przypadkach powinniśmy alarmować w pierwszej kolejności. Po pierwsze, ju na etapie wprowadzania do systemu, a po drugie do ewentualnej kontroli i ponownego liczenia glosów

## Sumowanie anomalii

### Na korzyść Trzaskowskiego

In [32]:
df[cand_A + '_sum_anomalies'] = df[[
    cand_A + '_anomaly_1', 
    cand_A + '_anomaly_2',
    cand_A + '_anomaly_3',
    cand_A + '_anomaly_4']].sum(axis=1)

In [33]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_A, sum(df[cand_A + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
trzaskowski 8406
2 anomalies:
trzaskowski 962
3 anomalies:
trzaskowski 24
4 anomalies:
trzaskowski 2


vs 2025:

1 anomalies:
trzaskowski 8161


2 anomalies:
trzaskowski 1179


3 anomalies:
trzaskowski 34


4 anomalies:
trzaskowski 3

In [34]:
# Komisje z wszystkimi czterma anomaliami
df[df[cand_A + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,trzaskowski_r1,duda_r1,postal_code,trzaskowski_r2,duda_r2,postal_clean,bucket,trzaskowski_anomaly_1,duda_anomaly_1,trzaskowski_anomaly_2,duda_anomaly_2,trzaskowski_anomaly_3,duda_anomaly_3,trzaskowski_anomaly_4,duda_anomaly_4,trzaskowski_sum_anomalies
5955,81006,1,125,402,67-312,488,220,67312,2377,True,False,True,False,True,False,True,False,4
10728,140611,6,287,684,05-660,842,506,05660,94,True,False,True,False,True,False,True,False,4


### Na korzyść Nawrockiego

In [35]:
df[cand_B + '_sum_anomalies'] = df[[
    cand_B + '_anomaly_1', 
    cand_B + '_anomaly_2',
    cand_B + '_anomaly_3',
    cand_B + '_anomaly_4']].sum(axis=1)

In [37]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_B, sum(df[cand_B + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
duda 6243
2 anomalies:
duda 571
3 anomalies:
duda 47
4 anomalies:
duda 0


vs 2025:

1 anomalies:
nawrocki 6871


2 anomalies:
nawrocki 483


3 anomalies:
nawrocki 34


4 anomalies:
nawrocki 2

In [39]:
# dla Dudy z 2020 nie ma przypadków z 4 anomaliami
df[df[cand_B + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,trzaskowski_r1,duda_r1,postal_code,trzaskowski_r2,duda_r2,postal_clean,bucket,trzaskowski_anomaly_1,duda_anomaly_1,trzaskowski_anomaly_2,duda_anomaly_2,trzaskowski_anomaly_3,duda_anomaly_3,trzaskowski_anomaly_4,duda_anomaly_4,trzaskowski_sum_anomalies,duda_sum_anomalies
